### Detach one from another

In [1]:
import trimesh
import numpy as np
from utils import *
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
### Generating a sample mesh
"""
mesh = trimesh.load("data/shrink.ply")
connected_components = mesh.split(only_watertight=True)
centroids = [comp.centroid for comp in connected_components]
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance
connected_components[1].apply_translation(normalized_vector * 0.09)
new_mesh = trimesh.util.concatenate(connected_components)
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)
print(pymesh.detect_self_intersection(x))
pymesh.save_mesh("data/shrink2.ply", x)
"""


In [2]:
def trimesh_to_open3d(tri_mesh: trimesh.Trimesh) -> o3d.geometry.TriangleMesh:
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(tri_mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(tri_mesh.faces)
    return o3d_mesh

def pymesh_to_trimesh(mesh):
    return trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)

def trimesh_to_pymesh(mesh):
    return pymesh.form_mesh(vertices=mesh.vertices, faces=mesh.faces)

def open3d_to_pymesh(o3d_mesh: o3d.geometry.TriangleMesh) -> pymesh.Mesh:
    vertices = np.asarray(o3d_mesh.vertices)
    faces = np.asarray(o3d_mesh.triangles)
    return  pymesh.form_mesh(vertices, faces)


def open3d_to_trimesh(o3d_mesh: o3d.geometry.TriangleMesh) -> trimesh.Trimesh:
    vertices = np.asarray(o3d_mesh.vertices)
    faces = np.asarray(o3d_mesh.triangles)
    return trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

In [3]:
def shrink_inner_mesh_any_case(
    inner_o3d_mesh: o3d.geometry.TriangleMesh,
    outer_o3d_mesh: o3d.geometry.TriangleMesh,
    contact_threshold=0.06,
    max_iters=10,
    num_sample_points=50000
):
    """
    Iteratively shrinks inner mesh so it lies inside outer mesh

    1) sample the outer mesh.
    2) see which points are inside or outside.
    3) For each iteration:
       a) Find which vertices of inner mesh are outside -> pull them inward.
       b) For vertices that are inside but too close to the outer surface,
          also pull them inward slightly.
    """

    # Build a point cloud
    pcd_outer = outer_o3d_mesh.sample_points_poisson_disk(number_of_points=num_sample_points)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_outer)
    
    # Access the inner mesh vertices
    outer_tri = open3d_to_trimesh(outer_o3d_mesh)
    inner_verts = np.asarray(inner_o3d_mesh.vertices)
    center_inner = np.mean(inner_verts, axis=0)

    for _iter in range(max_iters):
        changed_any = False

        # Check which vertices are inside vs. outside
        inside_mask = outer_tri.contains(inner_verts)
        
        # Iterate over each vertex
        for i in range(len(inner_verts)):
            v = inner_verts[i]

            # Perform KD-tree search once
            k, idx, dist_sq = pcd_tree.search_knn_vector_3d(v, 1)
            if k == 0:
                continue  # No nearest point found, skip

            dist = np.sqrt(dist_sq[0])  # Distance to closest surface

            direction = v - center_inner
            length = np.linalg.norm(direction)
            if length > 1e-12:
                direction_unit = direction / length

            if not inside_mask[i]:
                offset = dist + contact_threshold
                if offset > 0:
                    inner_verts[i] = v - offset * direction_unit
                    changed_any = True
                continue

            # If inside, check distance to nearest surface point
            if dist < contact_threshold:
                # Pull inward so that it is at least contact_threshold from the outer surface
                offset = contact_threshold - dist
                if offset > 0:
                    inner_verts[i] = v - offset * direction_unit
                    changed_any = True

        if not changed_any:
            # No vertex changed => we've converged
            break

    inner_o3d_mesh.vertices = o3d.utility.Vector3dVector(inner_verts)
    return inner_o3d_mesh



def detach_inner_component_general(
    input_path,
    contact_threshold=0.06,
    max_iters=10,
    num_sample_points=50000
):
    """
    1) Load a mesh and split into submeshes.
    2) Identify smallest-volume piece as 'inner', largest-volume piece as 'outer'.
    3) Convert each to Open3D. 
    4) Iteratively shrink the inner mesh so it stays inside the outer.
    """
    tm = trimesh.load(input_path, process=False)
    submeshes = tm.split(only_watertight=True)
    
    # Sort by volume: smallesr -> "inner", larger -> "outer"
    submeshes_sorted = sorted(submeshes, key=lambda m: m.volume)
    mesh_inner_tm = submeshes_sorted[0]
    mesh_outer_tm = submeshes_sorted[-1]

    # Convert to Open3D
    mesh_inner_o3d = trimesh_to_open3d(mesh_inner_tm)
    mesh_outer_o3d = trimesh_to_open3d(mesh_outer_tm)

    # Compute normals for better sampling
    mesh_inner_o3d.compute_vertex_normals()
    mesh_outer_o3d.compute_vertex_normals()

    # Shrink the inner mesh
    mesh_inner_o3d = shrink_inner_mesh_any_case(
        inner_o3d_mesh=mesh_inner_o3d,
        outer_o3d_mesh=mesh_outer_o3d,
        contact_threshold=contact_threshold,
        max_iters=max_iters,
        num_sample_points=num_sample_points
    )

    combined = mesh_inner_o3d + mesh_outer_o3d
    return combined

In [4]:
input_ply = "data/shrink2.ply"

final2 = detach_inner_component_general(
    input_path=input_ply,
    contact_threshold=0.06,
    max_iters=10
)

original2 = pymesh.load_mesh(input_ply)
final2 = open3d_to_pymesh(final2)
print(pymesh.detect_self_intersection(original2))
print(pymesh.detect_self_intersection(final2))

[[ 828  104]
 [ 839  104]
 [ 471 1203]
 [ 471  989]
 [ 839  471]
 [1204  471]
 [ 256  989]
 [ 622  989]
 [ 622 1355]
 [ 171 1365]
 [ 538 1365]
 [ 622  999]
 [ 622 1365]
 [ 363 1365]
 [ 548 1365]
 [1000  548]
 [ 998  548]
 [1364  548]
 [1376  363]
 [1010  548]
 [1376  548]
 [  43 1289]
 [ 615 1289]
 [ 460 1195]
 [ 410 1289]
 [  23  829]
 [ 408  829]
 [  66  829]
 [ 388  829]
 [ 388 1195]
 [  43 1376]
 [ 546 1376]
 [ 615 1376]
 [ 460  828]
 [ 818   23]
 [ 820   23]
 [1185   23]
 [ 189  932]
 [ 556  932]
 [ 189 1289]
 [1294  189]
 [1294  556]
 [ 928  189]
 [ 929  189]
 [1295  189]
 [1174   23]
 [1103  380]
 [1174  380]
 [ 553  929]
 [  13  737]
 [  13 1103]
 [ 386  737]
 [1291  178]
 [ 920  178]
 [1291  553]
 [ 738   12]
 [ 738  386]
 [1104  386]
 [1100   12]
 [ 379 1100]
 [ 545  920]
 [ 187  920]
 [ 379  734]
 [ 187 1286]
 [  19  734]
 [1101   19]
 [ 735  374]
 [1101  374]
 [1187  374]
 [ 174  916]
 [ 550  916]
 [ 550 1286]
 [ 909  174]
 [1283  174]
 [1274  174]
 [ 905  166]
 [ 905  174]

In [5]:
visualize_two_meshes(original2, final2)

Widget(value='<iframe src="http://localhost:39941/index.html?ui=P_0x7f7405c501f0_0&reconnect=auto" class="pyvi…

None